In [ ]:
!pip install transformers datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9a4c276c1db4a9bd1601b4947d3066e8f9104795f12182b4c627ee5d38316286
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's d

The original XSum dataset consists of 204,045 training samples, 11,332 validation samples, and 11,334 test samples. I have reduced it to 2,000 training samples and 500 samples each for validation and testing.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("xsum")

train_dataset = dataset["train"].select(range(2000))
val_dataset = dataset["validation"].select(range(500))
test_dataset = dataset["test"].select(range(500))

example = train_dataset[0]
print("Article:", example["document"])
print("Summary:", example["summary"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


(…)SUM-EMNLP18-Summary-Data-Original.tar.gz:   0%|          | 0.00/255M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Article: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small", truncation=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
prefix = "summarize: "
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small", return_tensors="tf")


In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)


In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    train_dataset,
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_val_set = model.prepare_tf_dataset(
    val_dataset,
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    test_dataset,
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)


In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
model.fit(tf_train_set, validation_data=tf_val_set, epochs=20)

Epoch 1/20
250/250 [==============================] - 44s 87ms/step - loss: 3.2467 - val_loss: 2.8173
Epoch 2/20
250/250 [==============================] - 18s 74ms/step - loss: 2.9793 - val_loss: 2.7358
Epoch 3/20
250/250 [==============================] - 18s 73ms/step - loss: 2.9062 - val_loss: 2.7023
Epoch 4/20
250/250 [==============================] - 18s 73ms/step - loss: 2.8513 - val_loss: 2.6808
Epoch 5/20
250/250 [==============================] - 18s 74ms/step - loss: 2.8165 - val_loss: 2.6671
Epoch 6/20
250/250 [==============================] - 18s 74ms/step - loss: 2.7835 - val_loss: 2.6560
Epoch 7/20
250/250 [==============================] - 18s 74ms/step - loss: 2.7490 - val_loss: 2.6476
Epoch 8/20
250/250 [==============================] - 19s 74ms/step - loss: 2.7255 - val_loss: 2.6423
Epoch 9/20
250/250 [==============================] - 18s 73ms/step - loss: 2.6946 - val_loss: 2.6383
Epoch 10/20
250/250 [==============================] - 18s 73ms/step - loss: 2.669

In [ ]:
from transformers import pipeline
text = """
summarize: Scientists have developed a new method to predict earthquakes by analyzing seismic activity in real-time.
The breakthrough could help save countless lives by providing accurate early warnings.
"""
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
summarizer(text)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 200, but your input_length is only 42. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


[{'summary_text': 'Scientists have developed a new method to predict earthquakes by analysing seismic activity in real-time - saving lives by providing accurate early warnings.'}]

In [ ]:
from evaluate import evaluator

task_evaluator = evaluator("summarization")

results = task_evaluator.compute(
    model_or_pipeline=summarizer,
    data=test_dataset.select(range(16)),
    input_column="document",
    label_column="summary",
    metric="rouge",
)


Your max_length is set to 200, but your input_length is only 65. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=32)
Your max_length is set to 200, but your input_length is only 69. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)
Your max_length is set to 200, but your input_length is only 155. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=77)
Your max_length is set to 200, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)


In [ ]:
print(results)

{'rouge1': 0.25265680065519797, 'rouge2': 0.07195203751256563, 'rougeL': 0.18104856178673776, 'rougeLsum': 0.18081710193415743, 'total_time_in_seconds': 156.88683424399994, 'samples_per_second': 0.10198433843795858, 'latency_in_seconds': 9.805427140249996}


The rouge scores show some overlap,Rouge-1 (25.3%), Rouge-2 (7.2%), rouge-L (18.1%). Processing took 157 seconds, with slower output speed, showing the content matches okay but could be better.